In [1]:
from time import sleep

import numpy as np

import cv2

from keras.models import load_model
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image

In [8]:
#LOADING HAAR CASCADE TRAINED MODEL

#CHANGE PATH FOR YOUR COMPUTER
xml_path = r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector\haarcascade.xml'
#CHANGE PATH FOR YOUR COMPUTER
model_path = r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector\Emo_little-h5.h5'

face_classifier = cv2.CascadeClassifier(xml_path)
classifier = load_model(model_path)

In [9]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 36EF-043D

 Directory of C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector

05/26/2021  03:53 PM    <DIR>          .
05/26/2021  03:53 PM    <DIR>          ..
04/06/2021  07:48 AM             1,799 .gitignore
04/11/2021  04:55 AM    <DIR>          .ipynb_checkpoints
04/06/2021  07:48 AM    <DIR>          emoji
04/11/2021  04:29 AM            27,308 Emotion_Detector.ipynb
05/26/2021  03:53 PM             6,781 Emotion_Detector_v2.ipynb
04/06/2021  07:48 AM        16,085,848 Emo_little-h5.h5
04/06/2021  07:48 AM           963,441 haarcascade.xml
04/06/2021  07:48 AM             1,706 Testing.py
04/06/2021  07:48 AM             4,710 Training.py
               7 File(s)     17,091,593 bytes
               4 Dir(s)  58,559,193,088 bytes free


In [10]:
#CHANGE PATH FOR YOUR COMPUTER
emoji_folder = r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector\emoji'

In [11]:
#LABELS FOR PREDICTION
class_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [12]:
#LOADING EMOJI IMAGES FROM EMOJI FOLDER
angry_filter = cv2.imread(emoji_folder + '/angry.png', -1)
happy_filter = cv2.imread(emoji_folder + '/happy.png', -1)
neutral_filter = cv2.imread(emoji_folder + '/neutral.png', -1)
sad_filter = cv2.imread(emoji_folder + '/sad.png', -1)
surprised_filter = cv2.imread(emoji_folder + '/surprised.png', -1)

In [13]:
#INITIATING VIDEOCAMERA
cap = cv2.VideoCapture(0)

In [14]:
#### WORKING CODE for every task
# NOT ADDED FOR NEUTRAL SO THAT CLEAR DIFFERENCE BETWEEN EMOTIONS CAN BE OBSERVED
while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    img_copy = np.copy(frame)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
        
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            label_position = (x, y)
            
            img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)
            
            #  FILTER APPLICATION BASED ON PREDICTED LABEL 
            if label == 'Surprise':
                filter_frame = cv2.resize( surprised_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Happy':
                filter_frame = cv2.resize( happy_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Angry':
                filter_frame = cv2.resize( angry_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Sad':
                filter_frame = cv2.resize( sad_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
            
            #Displaying label name on the face
            cv2.putText(img_copy, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            
            #Displaying Region of the face
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
        else:
            cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    
    #DISPLAYING THE VIDEO
    cv2.imshow('Emotion Detector', img_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# press q for exiting the window

# Restart Kernal before running again, else you'll face erros !!
(due to webcam allotment issues at OS level